## Welcome to C# notebooks in Azure Cosmos DB!## 

### Use Cosmos .NET SDK to create resources ###

In [3]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

// Create a new database and container with 400 RU/s
Microsoft.Azure.Cosmos.Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("Demo");
Container container = await database.CreateContainerIfNotExistsAsync("PaymentData", "/id", 400);

Display.AsMarkdown(@"
Created database Demo and container PaymentData.
");


Created database Demo and container PaymentData.


### Represent C# classes ###

In [8]:
using System;

class PaymentEvent
{
    public string id { get; set; }

    public Decimal Amount { get; set; }

    public string TransactionType { get; set; }

    public string Currency { get; set; }

    public string Username { get; set; }

    public string Country { get; set; }

    public string Address { get; set; }

    public string Date { get; set; }

    public DateTime Timestamp { get; set; }

}

### Generate some data ###

#### Use custom NuGet packages ####

In [9]:
#r "nuget: Bogus"
#r "nuget: System.Linq"
#r "nuget: System.Linq.Expressions"

In [10]:
using System.Linq;
using System.Collections.Generic;
using System.Linq.Expressions;
using System;
using Bogus;

static List<PaymentEvent> GenerateRandomPaymentEvents(int numberOfDocumentsPerBatch)
{
    var faker = new Faker();

    var paymentEvent = new Faker<PaymentEvent>()
        .StrictMode(true)
        //Generate event
        .RuleFor(o => o.id, f => Guid.NewGuid().ToString())
        .RuleFor(o => o.Amount, f => f.Finance.Amount())
        .RuleFor(o => o.TransactionType, f => f.Finance.TransactionType())
        .RuleFor(o => o.Currency, f => f.Finance.Currency().ToString())
        .RuleFor(o => o.Username, f => f.Internet.UserName())
        .RuleFor(o => o.Country, f => f.Address.Country())
        .RuleFor(o => o.Address, f => f.Address.StreetAddress())
        .RuleFor(o => o.Timestamp, f => DateTime.Now) // just today's date
        .RuleFor(o => o.Date, (f, m) => $"{m.Timestamp.ToString("yyyy-MM-dd")}");

    var events = paymentEvent.Generate(numberOfDocumentsPerBatch);
    
    return events;
    
}

#### Write data to PaymentData container ####

In [4]:
var paymentEvents = GenerateRandomPaymentEvents(100);

foreach (PaymentEvent paymentEvent in paymentEvents)
{
    var itemResponse = await container.CreateItemAsync<PaymentEvent>(paymentEvent);
}

Display.AsMarkdown(@"
Created 100 items in PaymentData container. 
");


Created 100 items in PaymentData container. 


### Query your data ###

In [13]:
using System.Linq;

QueryDefinition queryDefinition = new QueryDefinition("SELECT * FROM c WHERE c.TransactionType IN ('deposit', 'invoice', 'withdrawal')");

FeedIterator<PaymentEvent> queryResultSetIterator = container.GetItemQueryIterator<PaymentEvent>(queryDefinition);

List<PaymentEvent> paymentEvents = new List<PaymentEvent>();

while (queryResultSetIterator.HasMoreResults)
{
    FeedResponse<PaymentEvent> currentResultSet = await queryResultSetIterator.ReadNextAsync();
    foreach (PaymentEvent paymentEvent in currentResultSet)
    {
        paymentEvents.Add(paymentEvent);
    }
}

paymentEvents.Take(20)

System.Linq.Enumerable+ListPartition`1[Submission#11+PaymentEvent]

Message: 